In [1]:
#Connecting to the CMC api
#Imports for needed tools
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint
import pandas as pd
import requests
#Setup needed header, params and pp 
pp = pprint.PrettyPrinter(indent=4)
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY':'c01ce4f8-7e2a-421c-87fc-a68b52ab7952',
}

session = Session()
session.headers.update(headers)
#Use try to test the response and see if our get is functioning
try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  pp.pprint("success")
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)


#transformation 0) The data is in json so we need to snag it and load it into a dataframe
response2 = requests.request("GET", url, params=parameters, headers=headers).json()
df = pd.DataFrame(response2['data'])
df.to_csv('response_python.csv')


'success'


In [2]:
#Check out the df we are working in.
print(df.shape)
df.head()
# Transformation 1)Id the first set of headers we could use.
df2= df[['name','date_added','symbol','tags']]
df2.shape

#So the tags header is a mess... we need the information from it but we have a list the column. This make anaylsis rough.
# Transformation 2) We use .apply(pd.Series) to create a seperate df of the tags. We are luck the columns were setup to contain the info we need
# in the first two items in the list.
df2e = df2["tags"].apply(pd.Series)
print(df2e.shape)
#Test snagging a column by index
df2er= df2e.iloc[:,0]
#Transformation 3)Add the indexed columns to our df2
df2['type']=df2e.loc[:,0]
df2['concensus']=df2e.loc[:,1]
#Transformation 4)nuke the messy tag columns
df2=df2.drop('tags',axis=1)
print(df2.shape)
#We have a semi clean df2 now
print(df2.head())
#I want to clean up the timecode a bit for transformation 5 we do not need information to the second for our setup.

df2['date_added']=df2['date_added'].str[:-14]

df2.head()

(5000, 18)
(5000, 29)
(5000, 5)
          name                date_added symbol                type  \
0      Bitcoin  2010-07-13T00:00:00.000Z    BTC            mineable   
1     Ethereum  2015-08-07T00:00:00.000Z    ETH                 pos   
2  Tether USDt  2015-02-25T00:00:00.000Z   USDT            payments   
3          BNB  2017-07-25T00:00:00.000Z    BNB         marketplace   
4          XRP  2013-08-04T00:00:00.000Z    XRP  medium-of-exchange   

              concensus  
0                   pow  
1       smart-contracts  
2            stablecoin  
3  centralized-exchange  
4  enterprise-solutions  


<ipython-input-2-d577c49e29c3>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['type']=df2e.loc[:,0]
<ipython-input-2-d577c49e29c3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['concensus']=df2e.loc[:,1]


,name,date_added,symbol,type,concensus
0,Bitcoin,2010-07-13,BTC,mineable,pow
1,Ethereum,2015-08-07,ETH,pos,smart-contracts
2,Tether USDt,2015-02-25,USDT,payments,stablecoin
3,BNB,2017-07-25,BNB,marketplace,centralized-exchange
4,XRP,2013-08-04,XRP,medium-of-exchange,enterprise-solutions
